# Training and Deploying a DeepAR Model

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
import sagemaker 
import boto3
from sagemaker import get_execution_role 

role = get_execution_role()
session = sagemaker.Session()
region_name = boto3.Session().region_name

In [ ]:
%store -r s3_bucket
%store -r prefix

In [ ]:
training_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/training.jsonl"
test_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/test.jsonl"
training_s3_output_location = f"s3://{s3_bucket}/{prefix}/output/"

In [ ]:
from sagemaker.inputs import TrainingInput
train = TrainingInput(
    training_s3_input_location, 
    content_type="json"
)
test = TrainingInput(
    test_s3_input_location, 
    content_type="json"
)

In [ ]:
%store -r prediction_length
prediction_length

In [ ]:
%store -r freq
freq

In [ ]:
context_length = prediction_length

In [ ]:
from sagemaker.image_uris import retrieve 
container = retrieve(
    "forecasting-deepar", 
    region_name, 
    "1"
)
container

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.c4.2xlarge',
    output_path=training_s3_output_location,
    sagemaker_session=session
)

In [ ]:
estimator.set_hyperparameters(
    time_freq=freq,
    context_length=str(context_length),
    prediction_length=str(prediction_length),
    num_cells=40,
    num_layers=3,
    likelihood="gaussian",
    epochs=20,
    mini_batch_size=32,
    learning_rate=0.001,
    dropout_rate=0.05,
    early_stopping_patience=10
)

In [ ]:
data_channels = {"train": train, "test": test}

In [ ]:
%%time

estimator.fit(inputs=data_channels)

In [ ]:
estimator.model_data

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge"
)

In [ ]:
endpoint_name = predictor.endpoint_name

In [ ]:
%store endpoint_name